In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

# import wandb

In [2]:
cfg = Config.fromfile('./_swin_faster_config.py')

In [3]:
dataset = build_dataset(cfg.data.train)

loading annotations into memory...
Done (t=0.20s)
creating index...
index created!


In [4]:
dataset


CocoDataset Train dataset with number of images 4883, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Paper pack]  | 897   | 3 [Metal]   | 936   | 4 [Glass]    | 982   |
| 5 [Plastic]       | 2943  | 6 [Styrofoam] | 1263  | 7 [Plastic bag] | 5178  | 8 [Battery] | 159   | 9 [Clothing] | 468   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [5]:
dataset.pipeline

Compose(
    LoadImageFromFile(to_float32=False, color_type='color', file_client_args={'backend': 'disk'})
    LoadAnnotations(with_bbox=True, with_label=True, with_mask=False, with_seg=False, poly2mask=True, poly2mask={'backend': 'disk'})
    Resize(img_scale=[(1024, 1024)], multiscale_mode=range, ratio_range=None, keep_ratio=True, bbox_clip_border=True)
    RandomFlip(flip_ratio=0.5)
    Normalize(mean=[123.675 116.28  103.53 ], std=[58.395 57.12  57.375], to_rgb=True)
    Pad(size=None, size_divisor=32, pad_to_square=False, pad_val={'img': 0, 'masks': 0, 'seg': 255})
    DefaultFormatBundle(img_to_float=True)
    Collect(keys=['img', 'gt_bboxes', 'gt_labels'], meta_keys=('filename', 'ori_filename', 'ori_shape', 'img_shape', 'pad_shape', 'scale_factor', 'flip', 'flip_direction', 'img_norm_cfg'))
)

In [6]:
next(iter(dataset))

{'img_metas': DataContainer({'filename': '../../dataset/train/0000.jpg', 'ori_filename': 'train/0000.jpg', 'ori_shape': (1024, 1024, 3), 'img_shape': (1024, 1024, 3), 'pad_shape': (1024, 1024, 3), 'scale_factor': array([1., 1., 1., 1.], dtype=float32), 'flip': False, 'flip_direction': None, 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32), 'std': array([58.395, 57.12 , 57.375], dtype=float32), 'to_rgb': True}}),
 'img': DataContainer(tensor([[[-0.6965, -0.4568, -0.4054,  ..., -0.9192, -0.9192, -0.7650],
          [ 0.8447,  0.1597, -0.6452,  ..., -0.7822, -0.8164, -0.7308],
          [ 0.6906,  0.5878, -0.2342,  ..., -0.6794, -0.7308, -0.7137],
          ...,
          [ 0.0056, -0.6794, -0.6965,  ..., -0.8678, -0.8678, -0.8849],
          [-0.2342, -0.7479, -0.6281,  ..., -0.7650, -0.8507, -0.9192],
          [-0.5253, -1.0219, -0.7650,  ..., -0.6794, -0.8507, -0.9877]],
 
         [[-0.7052, -0.4601, -0.3725,  ..., -0.7402, -0.7402, -0.5826],
          [ 0.8

In [7]:
model = build_detector(cfg.model)
model.init_weights()
print(model)

2022-04-05 10:25:19,306 - mmdet - INFO - load checkpoint from local path: ./swin_large_patch4_window7_224_22kto1k.pth
2022-04-05 10:25:20,234 - mmdet - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2022-04-05 10:25:20,285 - mmdet - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-04-05 10:25:20,302 - mmdet - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override': [{'name': 'shared_fcs'}, {'name': 'cls_fcs'}, {'name': 'reg_fcs'}]}]
2022-04-05 10:25:20,474 - mmcv - INFO - 
backbone.patch_embed.projection.weight - torch.Size([192, 3, 4, 4]): 
Initialized by user-defined `init_weights` in SwinTransformer  
 
2022-04-05 10:25:20,475 - mmcv - INFO - 
backbone.patch_embed.projection.bias - torch.Size([192]): 
In

FasterRCNN(
  (backbone): SwinTransformer(
    (patch_embed): PatchEmbed(
      (adap_padding): AdaptivePadding()
      (projection): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
      (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
    )
    (drop_after_pos): Dropout(p=0.0, inplace=False)
    (stages): ModuleList(
      (0): SwinBlockSequence(
        (blocks): ModuleList(
          (0): SwinBlock(
            (norm1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
            (attn): ShiftWindowMSA(
              (w_msa): WindowMSA(
                (qkv): Linear(in_features=192, out_features=576, bias=True)
                (attn_drop): Dropout(p=0.0, inplace=False)
                (proj): Linear(in_features=192, out_features=192, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
                (softmax): Softmax(dim=-1)
              )
              (drop): DropPath()
            )
            (norm2): LayerNorm((192,), eps=1e-05, elemen

In [8]:
train_detector(model, dataset, cfg, distributed=False, validate=False)

2022-04-05 10:25:25,858 - mmdet - INFO - Start running, host: root@2603c2edf705, work_dir: /opt/ml/detection/baseline/mmdetection/work_dirs/_swin_faster
2022-04-05 10:25:25,859 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(ABOVE_NORMAL) Fp16OptimizerHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) Fp16OptimizerHook                  
(NORMAL      ) CheckpointHook                     
(LOW     